## Imports and Setup

In [1]:
from fredapi import Fred
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
import plotly.express as px
import json

load_dotenv()

FRED_API_KEY = os.getenv("FRED_API_KEY") 

fred = Fred(api_key=FRED_API_KEY)

In [2]:
def series_to_json(series, df):
    """
    Convert a FRED series to a JSON string.
    """
    path = f"datasets/fred_{series}.json"
    # If file exists, load and merge
    if os.path.exists(path):
        # Load existing data
        with open(path, "r") as f:
            existing_data = json.load(f)
        df_existing = pd.DataFrame(existing_data)
        df_existing["Date"] = pd.to_datetime(df_existing["Date"])

        # Append only new rows
        df_combined = pd.concat([df_existing, df])
        df_combined = df_combined.drop_duplicates(subset=["Date"]).sort_values("Date")
    else:
        # No file exists, use fresh data
        df_combined = df

    # Save to JSON
    df_combined.to_json(path, orient="records", date_format="iso")

    print(f"Updated data saved to {path}")

## S&P 500 - Daily

In [20]:
spdata = fred.get_series('SP500')
df_spdata = spdata.to_frame().reset_index()
df_spdata.columns = ['Date', 'Price']

series_to_json('SP500', df_spdata)

spdata.tail()

Updated data saved to datasets/fred_SP500.json


2025-08-04    6329.94
2025-08-05    6299.19
2025-08-06    6345.06
2025-08-07    6340.00
2025-08-08    6389.45
dtype: float64

In [ ]:
spfig = px.line(df_spdata, x='Date', y='Price', title='S&P 500 Index Over Time')
spfig.show()

## M2 - Monthly

In [ ]:
dataset_id = "M2SL"

m2 = fred.get_series(dataset_id)
df_m2 = m2.to_frame().reset_index()
df_m2.columns = ['Date', 'M2']

# # If file exists, load and merge
# if os.path.exists(json_path):
#     # Load existing data
#     with open(json_path, "r") as f:
#         existing_data = json.load(f)
#     df_existing = pd.DataFrame(existing_data)
#     df_existing["Date"] = pd.to_datetime(df_existing["Date"])

#     # Append only new rows
#     df_combined = pd.concat([df_existing, df_m2])
#     df_combined = df_combined.drop_duplicates(subset=["Date"]).sort_values("Date")
# else:
#     # No file exists, use fresh data
#     df_combined = df_m2

# # Save to JSON
# df_combined.to_json(json_path, orient="records", date_format="iso")

# print(f"Updated data saved to {json_path}")

series_to_json(dataset_id, df_m2)
df_m2.tail()

Updated data saved to datasets/fred_M2SL.json


,Date,M2
793,2025-02-01,21584.1
794,2025-03-01,21656.9
795,2025-04-01,21804.5
796,2025-05-01,21883.6
797,2025-06-01,22020.8


In [4]:
m2fig = px.line(df_m2, x='Date', y='M2', title='M2 Money Stock Over Time')
m2fig.show()

## M2 Velocity - 3mos

In [ ]:
dataset_id = "M2V"
m2v = fred.get_series(dataset_id)
df_m2v = m2v.to_frame().reset_index()
df_m2v.columns = ['Date', 'M2 Velocity']

series_to_json('M2V', df_m2v)

df_m2v.tail()

Updated data saved to datasets/fred_M2V.json


,Date,M2 Velocity
261,2024-04-01,1.382
262,2024-07-01,1.387
263,2024-10-01,1.389
264,2025-01-01,1.388
265,2025-04-01,1.385


## M2 Stock: Nominal vs Real

In [ ]:
series = {
    "M2 Money Stock (Nominal)": "M2SL",
    "M2 Money Stock (Real)": "M2REAL"
}

# Download series
data = {name: fred.get_series(code) for name, code in series.items()}

# Combine into a single DataFrame
df = pd.DataFrame(data)
df.index.name = "Date"

# Trim to shared date range
df.dropna(inplace=True)

# Plot
plt.figure(figsize=(12, 6))
plt.plot(df.index, df["M2 Money Stock (Nominal)"], label="Nominal M2", linewidth=2)
plt.plot(df.index, df["M2 Money Stock (Real)"], label="Real M2 (Inflation-adjusted)", linewidth=2)
plt.title("M2 Money Stock: Nominal vs Real", fontsize=16)
plt.xlabel("Year")
plt.ylabel("Billions of Dollars")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## M2 Supply + Velocity

In [ ]:
# Get Data
real_m2 = fred.get_series("M2REAL")
velocity = fred.get_series("M2V")
recessions = fred.get_series("USREC")

# Combine into one DataFrame
df = pd.concat([real_m2, velocity, recessions], axis=1)
df.columns = ["Real M2", "Velocity of M2", "USREC"]
df.dropna(inplace=True)

# Plot
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot Real M2 on left axis
ax1.set_xlabel("Year")
ax1.set_ylabel("Real M2 (Billions)", color="tab:blue")
ax1.plot(df.index, df["Real M2"], color="tab:blue", label="Real M2", linewidth=2)
ax1.tick_params(axis="y", labelcolor="tab:blue")

# Create second y-axis for velocity
ax2 = ax1.twinx()
ax2.set_ylabel("Velocity of M2", color="tab:red")
ax2.plot(df.index, df["Velocity of M2"], color="tab:red", label="Velocity of M2", linewidth=2)
ax2.tick_params(axis="y", labelcolor="tab:red")

# Shade recessions
for i in range(len(df)):
    if df["USREC"].iloc[i] == 1:
        ax1.axvspan(df.index[i], df.index[i], color="gray", alpha=0.3)

# Titles and legend
plt.title("Real M2 Stock vs. Velocity of M2 (with Recessions)", fontsize=12)
fig.tight_layout()
plt.show()


## Recessions - Monthly

In [ ]:
rec = fred.get_series('USREC')
df_rec = rec.to_frame().reset_index()
df_rec.columns = ['Date', 'USREC']

series_to_json('USREC', df_rec)

rec.tail()

Updated data saved to datasets/fred_USREC.json


2025-03-01    0.0
2025-04-01    0.0
2025-05-01    0.0
2025-06-01    0.0
2025-07-01    0.0
dtype: float64

In [9]:
rec_fig = px.line(df_rec, x='Date', y='USREC', title='US Recession Indicator')
rec_fig.update_layout(yaxis=dict(tickvals=[0, 1], ticktext=['Expansion', 'Recession']))
rec_fig.show()

## CPI - Monthly

In [ ]:
dataset_id = "CPIAUCSL"
cpi = fred.get_series(dataset_id)
df_cpi = cpi.to_frame().reset_index()
df_cpi.columns = ['Date', 'CPI']

series_to_json('CPIAUCSL', df_cpi)
df_cpi.tail()

Updated data saved to datasets/fred_CPIAUCSL.json


,Date,CPI
937,2025-02-01,319.775
938,2025-03-01,319.615
939,2025-04-01,320.321
940,2025-05-01,320.580
941,2025-06-01,321.500


In [45]:
cpifig = px.line(df_cpi, x='Date', y='CPI', title='Consumer Price Index (CPI) Over Time')
cpifig.show()

## PPI - Monthly

In [43]:
ppi = fred.get_series('PPIACO')
df_ppi = ppi.to_frame().reset_index()
df_ppi.columns = ['Date', 'PPI']

series_to_json('PPIACO', df_ppi)
df_ppi.tail()

Updated data saved to datasets/fred_PPIACO.json


,Date,PPI
1345,2025-02-01,259.498
1346,2025-03-01,258.507
1347,2025-04-01,258.298
1348,2025-05-01,258.501
1349,2025-06-01,260.180


In [ ]:
ppi_fig = px.line(df_ppi, x='Date', y='PPI', title='Producer Price Index (PPI) Over Time')
ppi_fig.update_layout(yaxis_title='PPI Index (1982=100)', xaxis_title='Date')
ppi_fig.show()

## PCE - Monthly

In [21]:
dataset_id = "PCE"

pce = fred.get_series(dataset_id)
df_pce = pce.to_frame().reset_index()
df_pce.columns = ['Date', 'PCE']

series_to_json('PCE', df_pce)
df_pce.tail()

Updated data saved to datasets/fred_PCE.json


,Date,PCE
793,2025-02-01,20436.3
794,2025-03-01,20578.5
795,2025-04-01,20622.3
796,2025-05-01,20615.2
797,2025-06-01,20685.2


In [22]:
pce_fig = px.line(df_pce, x='Date', y='PCE', title='Personal Consumption Expenditures (PCE) Over Time')
pce_fig.show()

## Fed Funds Rate - Monthly

In [ ]:
fedfunds = fred.get_series('FEDFUNDS')
df_fedfunds = fedfunds.to_frame().reset_index()
df_fedfunds.columns = ['Date', 'Fed Funds Rate']

series_to_json('FEDFUNDS', df_fedfunds)

df_fedfunds.tail()

Updated data saved to datasets/fred_FEDFUNDS.json


,Date,Fed Funds Rate
848,2025-03-01,4.33
849,2025-04-01,4.33
850,2025-05-01,4.33
851,2025-06-01,4.33
852,2025-07-01,4.33


In [12]:
fedfundsfig = px.line(df_fedfunds, x='Date', y='Fed Funds Rate', title='Federal Funds Rate Over Time')
fedfundsfig.show()

## Unemployment Rate  - Monthly

In [13]:
unrate = fred.get_series('UNRATE')
df_unrate = unrate.to_frame().reset_index()
df_unrate.columns = ['Date', 'Unrate']

series_to_json('UNRATE', df_unrate)
df_unrate.tail()

Updated data saved to datasets/fred_UNRATE.json


,Date,Unrate
926,2025-03-01,4.2
927,2025-04-01,4.2
928,2025-05-01,4.2
929,2025-06-01,4.1
930,2025-07-01,4.2


In [14]:
unrate_fig = px.line(df_unrate, x='Date', y='Unrate', title='Unemployment Rate Over Time')
unrate_fig.show()

### Average Earnings of All Employees, Total Private - Monthly

In [ ]:
hourly_earnings = fred.get_series('CES0500000003')
df_hourly_earnings = hourly_earnings.to_frame().reset_index()
df_hourly_earnings.columns = ['Date', 'Avg Hourly']
df_hourly_earnings['Date'] = pd.to_datetime(df_hourly_earnings['Date'])
# Add average yearly earnings
df_hourly_earnings['Avg Yearly'] = df_hourly_earnings['Avg Hourly'] * 40 * 52
df_hourly_earnings.tail()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Left axis: Hourly earnings
fig.add_trace(go.Scatter(
    x=df_hourly_earnings['Date'],
    y=df_hourly_earnings['Avg Hourly'],
    name='Avg Hourly Earnings',
    yaxis='y1',
    line=dict(color='blue')
))

# Right axis: Yearly earnings
fig.add_trace(go.Scatter(
    x=df_hourly_earnings['Date'],
    y=df_hourly_earnings['Avg Yearly'],
    name='Avg Yearly Earnings',
    yaxis='y2',
    line=dict(color='orange')
))

fig.update_layout(
    title='Average Hourly vs Yearly Earnings',
    xaxis=dict(title='Date'),
    yaxis=dict(
        title='Hourly Earnings (USD)',
        side='left'
    ),
    yaxis2=dict(
        title='Yearly Earnings (USD)',
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0.01, y=0.99)
)

fig.show()


## GDP - Quarterly

In [15]:
gdp = fred.get_series('GDP')
df_gdp = gdp.to_frame().reset_index()
df_gdp.columns = ['Date', 'GDP']

series_to_json('GDP', df_gdp)

df_gdp.tail()

Updated data saved to datasets/fred_GDP.json


,Date,GDP
313,2024-04-01,29016.714
314,2024-07-01,29374.914
315,2024-10-01,29723.864
316,2025-01-01,29962.047
317,2025-04-01,30331.117


In [ ]:
gdp_fig = px.line(df_gdp, x='Date', y='GDP', title='Gross Domestic Product (GDP) Over Time')
gdp_fig.update_layout(yaxis_title='GDP (Billions USD)', xaxis_title='Date')
gdp_fig.show()

## Real GDP - Quarterly

In [23]:
real_gdp = fred.get_series('GDPC1')
df_real_gdp = real_gdp.to_frame().reset_index()
df_real_gdp.columns = ['Date', 'Real GDP']

series_to_json('GDPC1', df_real_gdp)

df_real_gdp.tail()

Updated data saved to datasets/fred_GDPC1.json


,Date,Real GDP
309,2024-04-01,23223.906
310,2024-07-01,23400.294
311,2024-10-01,23542.349
312,2025-01-01,23512.717
313,2025-04-01,23685.287


In [24]:
real_gdp_fig = px.line(df_real_gdp, x='Date', y='Real GDP', title='Real Gross Domestic Product (GDP) Over Time')
real_gdp_fig.update_layout(yaxis_title='Real GDP (Billions USD)', xaxis_title='Date')
real_gdp_fig.show()

## S&P CoreLogic Case-Shiller U.S. National Home Price Index - Monthly

In [30]:
sp_case_shiller = fred.get_series('CSUSHPINSA')
df_sp_case_shiller = sp_case_shiller.to_frame().reset_index()
df_sp_case_shiller.columns = ['Date', 'CS Home Price Index']

series_to_json('CSUSHPINSA', df_sp_case_shiller)
df_sp_case_shiller.tail()

Updated data saved to datasets/fred_CSUSHPINSA.json


,Date,CS Home Price Index
600,2025-01-01,323.652
601,2025-02-01,325.107
602,2025-03-01,327.658
603,2025-04-01,329.638
604,2025-05-01,331.107


In [29]:
sp_case_fig = px.line(df_sp_case_shiller, x='Date', y='CS Home Price Index', title='S&P CoreLogic Case-Shiller U.S. National Home Price Index Over Time')
sp_case_fig.update_layout(yaxis_title='Home Price Index', xaxis_title='Date')
sp_case_fig.show()

## Median Sales Price of Houses in the US - Quarterly -> Monthly

In [16]:
median_home_prices = fred.get_series('MSPUS')
df_home_median_prices = median_home_prices.to_frame().reset_index()
df_home_median_prices.columns = ['Date', 'Median Sales Price']
df_home_median_prices['Date'] = pd.to_datetime(df_home_median_prices['Date'])
df_home_median_prices.set_index('Date', inplace=True)
df_home_median_monthly = df_home_median_prices.resample('ME').ffill()

series_to_json('MSPUS', df_home_median_monthly)

df_home_median_monthly.tail()

Updated data saved to datasets/fred_MSPUS.json


,Median Sales Price
Date,
2024-12-31,419300.0
2025-01-31,423100.0
2025-02-28,423100.0
2025-03-31,423100.0
2025-04-30,410800.0


## Average Sales Price of Houses in the US - Quarterly -> Monthly

In [17]:
avg_home_prices = fred.get_series('ASPUS')
df_home_avg_prices = avg_home_prices.to_frame().reset_index()
df_home_avg_prices.columns = ['Date', 'Average Sales Price']
df_home_avg_prices['Date'] = pd.to_datetime(df_home_avg_prices['Date'])
df_home_avg_prices.set_index('Date', inplace=True)
df_home_avg_monthly = df_home_avg_prices.resample('ME').ffill()

series_to_json('ASPUS', df_home_avg_monthly)

df_home_avg_monthly.tail(20)

Updated data saved to datasets/fred_ASPUS.json


,Average Sales Price
Date,
2023-09-30,521900.0
2023-10-31,498300.0
2023-11-30,498300.0
2023-12-31,498300.0
2024-01-31,519700.0
2024-02-29,519700.0
2024-03-31,519700.0
2024-04-30,502200.0
2024-05-31,502200.0


## Mortgage Rates - Weekly (ending Thurs)

### 30 Year Fixed

In [18]:
mtg30 = fred.get_series('MORTGAGE30US')
df_mtg30 = mtg30.to_frame().reset_index()
df_mtg30.columns = ['Date', '30yr']

series_to_json('MORTGAGE30US', df_mtg30)

df_mtg30.tail()

Updated data saved to datasets/fred_MORTGAGE30US.json


,Date,30yr
2832,2025-07-10,6.72
2833,2025-07-17,6.75
2834,2025-07-24,6.74
2835,2025-07-31,6.72
2836,2025-08-07,6.63


### 15 Year Fixed

In [19]:
mtg15 = fred.get_series('MORTGAGE15US')
df_mtg15 = mtg15.to_frame().reset_index()
df_mtg15.columns = ['Date', '15yr']

series_to_json('MORTGAGE15US', df_mtg15)

df_mtg15.tail()

Updated data saved to datasets/fred_MORTGAGE15US.json


,Date,15yr
1767,2025-07-10,5.86
1768,2025-07-17,5.92
1769,2025-07-24,5.87
1770,2025-07-31,5.85
1771,2025-08-07,5.75


### Combined Graph

In [25]:
mtgs_df = pd.merge(df_mtg30, df_mtg15, on='Date', how='inner')
mtgs_df.tail()

,Date,30yr,15yr
1767,2025-07-10,6.72,5.86
1768,2025-07-17,6.75,5.92
1769,2025-07-24,6.74,5.87
1770,2025-07-31,6.72,5.85
1771,2025-08-07,6.63,5.75


In [26]:
mtgs_melted = mtgs_df.melt(id_vars="Date", var_name='Mortgage Type', value_name='Rate (%)')
mtgs_melted.tail()

,Date,Mortgage Type,Rate (%)
3539,2025-07-10,15yr,5.86
3540,2025-07-17,15yr,5.92
3541,2025-07-24,15yr,5.87
3542,2025-07-31,15yr,5.85
3543,2025-08-07,15yr,5.75


In [27]:
# Plot
fig = px.line(
    mtgs_melted,
    x="Date",
    y="Rate (%)",
    color="Mortgage Type",
    title="US 30-Year vs 15-Year Fixed Mortgage Rates",
    labels={"Rate (%)": "Average Interest Rate (%)"}
)

fig.update_traces(line=dict(width=2))
fig.update_layout(template="plotly_white")

fig.show()

## Large Banks

### Consumer Mortgage Originations: New Originations - Quarterly

In [33]:
mtg_orig = fred.get_series('RCMFLOORIG')
df_mtg_orig = mtg_orig.to_frame().reset_index()
df_mtg_orig.columns = ['Date', 'New Originations']

series_to_json('RCMFLOORIG', df_mtg_orig)

df_mtg_orig.tail()

Updated data saved to datasets/fred_RCMFLOORIG.json


,Date,New Originations
46,2024-01-01,43.33
47,2024-04-01,59.64
48,2024-07-01,62.57
49,2024-10-01,67.98
50,2025-01-01,50.58


In [34]:
mtg_orig_fig = px.line(df_mtg_orig, x='Date', y='New Originations', title='Consumer Mortgage Originations: New Originations - Quarterly')
mtg_orig_fig.update_layout(yaxis_title='New Originations (Billions USD)', xaxis_title='Date')
mtg_orig_fig.show()

### Consumer Mortgage Originations: Number of New Accounts - Quarterly

In [39]:
mtg_orig_count = fred.get_series('RCMFLOACT')
df_mtg_orig_count = mtg_orig_count.to_frame().reset_index()
df_mtg_orig_count.columns = ['Date', 'New Accounts']

series_to_json('RCMFLOACT', df_mtg_orig_count)

df_mtg_orig_count.tail()

Updated data saved to datasets/fred_RCMFLOACT.json


,Date,New Accounts
46,2024-01-01,79061.0
47,2024-04-01,108278.0
48,2024-07-01,115373.0
49,2024-10-01,120851.0
50,2025-01-01,92690.0


In [40]:
mtg_orig_count_fig = px.line(df_mtg_orig_count, x='Date', y='New Accounts', title='Consumer Mortgage Originations: Number of New Accounts - Quarterly')
mtg_orig_count_fig.update_layout(yaxis_title='New Accounts (Count)', xaxis_title='Date')
mtg_orig_count_fig.show()

### Consumer Credit Card Balances: Total Balances - Quarterly

In [35]:
cc_balances = fred.get_series('RCCCBBALTOT')
df_cc_balances = cc_balances.to_frame().reset_index()
df_cc_balances.columns = ['Date', 'Total Balances']

series_to_json('RCCCBBALTOT', df_cc_balances)

df_cc_balances.tail()

Updated data saved to datasets/fred_RCCCBBALTOT.json


,Date,Total Balances
46,2024-01-01,881.23
47,2024-04-01,903.38
48,2024-07-01,913.86
49,2024-10-01,951.03
50,2025-01-01,908.42


In [36]:
cc_balances_fig = px.line(df_cc_balances, x='Date', y='Total Balances', title='Consumer Credit Card Balances: Total Balances - Quarterly')
cc_balances_fig.update_layout(yaxis_title='Total Balances (Billions USD)', xaxis_title='Date')
cc_balances_fig.show()

### Consumer Credit Card Balances: Total Purchase Volume - Quarterly

In [41]:
cc_purchase_vol = fred.get_series('RCCCBPURCHASETOT')
df_cc_purchase_vol = cc_purchase_vol.to_frame().reset_index()
df_cc_purchase_vol.columns = ['Date', 'Total Purchase Volume']

series_to_json('RCCCBPURCHASETOT', df_cc_purchase_vol)

df_cc_purchase_vol.tail()

Updated data saved to datasets/fred_RCCCBPURCHASETOT.json


,Date,Total Purchase Volume
46,2024-01-01,809.00
47,2024-04-01,878.51
48,2024-07-01,872.50
49,2024-10-01,908.12
50,2025-01-01,844.73


In [42]:
cc_purchase_vol_fig = px.line(df_cc_purchase_vol, x='Date', y='Total Purchase Volume', title='Consumer Credit Card Balances: Total Purchase Volume - Quarterly')
cc_purchase_vol_fig.update_layout(yaxis_title='Total Purchase Volume (Billions USD)', xaxis_title='Date')
cc_purchase_vol_fig.show()

### Consumer Credit Card Originations: New Originations - Quarterly

In [37]:
cc_originations = fred.get_series('RCCCOORG')
df_cc_originations = cc_originations.to_frame().reset_index()
df_cc_originations.columns = ['Date', 'New Originations']

series_to_json('RCCCOORG', df_cc_originations)

df_cc_originations.tail()

Updated data saved to datasets/fred_RCCCOORG.json


,Date,New Originations
46,2024-01-01,102.85
47,2024-04-01,106.78
48,2024-07-01,100.36
49,2024-10-01,102.04
50,2025-01-01,96.93


In [38]:
cc_originations_fig = px.line(df_cc_originations, x='Date', y='New Originations', title='Consumer Credit Card Originations: New Originations - Quarterly')
cc_originations_fig.update_layout(yaxis_title='New Originations (Billions USD)', xaxis_title='Date')
cc_originations_fig.show()